Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
val NAME = ""
val COLLABORATORS = ""

---

# Lab 4 - Functional Programming
> Labs will be due each week before the homeworks. They are not intended take a significant amount of time but rather to provide examples/practice on specific and isolated features in the language. Labs are autograded so you can get quick feedback.

### Import the necessary Chisel dependencies. 
> There will be a cell like this in every lab. Make sure you run it before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
val path = System.getProperty("user.dir") + "/resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chisel3.tester.RawTester.test

## Problem 1 (3 pts) - Seq addition
> Use Scala `zip` and `map` to add the contents two `Seq`s

In [ ]:
def addSeqs(a: Seq[Int], b: Seq[Int]): Seq[Int] = {
    // YOUR CODE HERE
    ???
}

In [ ]:
val a = Seq.tabulate(10)(_.toInt)
val b = Seq.tabulate(10)(_.toInt)
assert(addSeqs(a, b) == Seq(0, 2, 4, 6, 8, 10, 12, 14, 16, 18))


## Problem 2 (4 pts) - zipWithIndex
> Use `foldLeft` to implement the `exp` function (computes exponent). Then use `zipWithIndex`, `map`, and `exp` to consisely evaluate a polynomial. The index in the sequence is the degree in the polynomial (e.g. coefs(i) * x^i)

In [ ]:
def exp(base: Int, deg: Int): Int = {
    // YOUR CODE HERE
    ???
}

def polyEval(coefs: Seq[Int], x: Int): Int = {
    // YOUR CODE HERE
    ???
}

In [ ]:
assert (exp(5, 0) == 1)
assert (exp(2, 5) == 32)
assert (exp(4, 3) == 64)
// 0*x^0 + 1*x^1 + 2*x^2
assert(polyEval(Seq(0, 1, 2), 5) == 55)
assert(polyEval(Seq(0, 1, 2), 0) == 0)


## Problem 3 (5 pts) - map on matrix
> Given a `n` x `n` matrix, use `map` to add `x` to the diagonal (other cells unchanged). The matrix is in row-major order. For example if `x=4`: 
``` 
    List(1, 1, 1, 1, 1) -> List(5, 1, 1, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 5, 1, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 5, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 1, 5, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 1, 1, 5)
```

In [ ]:
// YOUR CODE HERE
???

In [ ]:
val in = List(
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1)
)

val out = List(
  List(5, 1, 1, 1, 1),
  List(1, 5, 1, 1, 1),
  List(1, 1, 5, 1, 1),
  List(1, 1, 1, 5, 1),
  List(1, 1, 1, 1, 5)
)
assert(incDiag(in, 4) == out)



## Problem 4 (10 pts) - Parameterized Vec ALU
> Let's put together what we've covered so far. The `VecOp` module will perform an operation on two `Vec`s specified by the `op` parameter. `op` can be three symbols: `+`, `-`, and `*`, corresponding to vector addition, subtraction, and multiplication (all with bit-growth). If a symbol other than the three specified is supplied, the module is expected to return a default `Vec` of `0.U`. Fill in the `testVecOp` function to test your work.

In [ ]:
class VecOp(n: Int, width: Int, op: String) extends Module {
    val io = IO(new Bundle {
        val a = Input(Vec(n, UInt(width.W)))
        val b = Input(Vec(n, UInt(width.W)))
        val c = Output(Vec(n, UInt()))
    })
    
    // YOUR CODE HERE
    ???
    
}

def testVecOp(n: Int, width: Int, op: String): Boolean = {
    test(new VecOp(n, width, op)) { dut =>
        // YOUR CODE HERE
        ???
    }
    true
}

In [ ]:

assert(testVecOp(4, 4, "+"))
assert(testVecOp(4, 4, "*"))
assert(testVecOp(4, 4, "-"))
assert(testVecOp(4, 2, "x"))